In [ ]:
import boto3 as boto
import time

In [ ]:
client = boto.client('ec2')

In [ ]:
response = client.run_instances(
    InstanceType = 't2.micro',
    MinCount = 1,
    MaxCount = 1,
    ImageId = 'ami-076e3a557efe1aa9c',
    KeyName = 'lab',
    SecurityGroupIds=['sg-076f3eb97b5647364'],
    UserData = '''#!/bin/bash
sudo yum update -y
sudo yum install -y httpd.x86_64 awscli
sudo systemctl start httpd.service
sudo systemctl enable httpd.service
usermod -a -G apache ec2-user
chown -R ec2-user:apache /var/www
chmod 2775 /var/www
mkdir ~/.aws
echo "[default]\nregion = ap-south-1" > ~/.aws/config
echo "[default]\naws_access_key_id = key\naws_secret_access_key = secret" > ~/.aws/credentials
aws s3 cp s3://xd-dead/ /var/www/html --recursive''')

In [ ]:
# print(response)

In [ ]:
response['Instances'][0]['InstanceId']

In [ ]:
desc = client.describe_instances(InstanceIds=[response['Instances'][0]['InstanceId']])
while(desc['Reservations'][0]['Instances'][0]["State"]["Name"]!='running'):
    time.sleep(5)
    desc = client.describe_instances(InstanceIds=[response['Instances'][0]['InstanceId']])
print(desc['Reservations'][0]['Instances'][0]['PublicDnsName'])